# Evaluation

In [1]:
# notebook_name = "2evaluation_sentiment+OCR"
notebook_name = "2evaluation_topics"

In [2]:
import define
import json
from os import listdir
import csv
import pandas as pd
from elasticsearch import Elasticsearch
import IPython
es = Elasticsearch(HOST="http://localhost", PORT=9200)
es = Elasticsearch()

In [3]:
# make dictionary with all topics
import xml.etree.ElementTree as ET
tree = ET.parse('topics.xml')
# tree = ET.parse('topics_selected.xml')
topics = tree.findall('topic')

topicsDic = {}

for topic in topics:
    title = topic.find('title').text
    number = topic.find('number').text
    topicsDic[number] = title

topicsDic

{'1': 'Should teachers get tenure?',
 '2': 'Is vaping with e-cigarettes safe?',
 '3': 'Should insider trading be allowed?',
 '4': 'Should corporal punishment be used in schools?',
 '5': 'Should social security be privatized?',
 '6': 'Is a college education worth it?',
 '7': 'Should felons who have completed their sentence be allowed to vote?',
 '8': 'Should abortion be legal?',
 '9': 'Should students have to wear school uniforms?',
 '10': 'Should any vaccines be required for children?',
 '11': 'Should performance-enhancing drugs be accepted in sports?',
 '12': 'Should birth control pills be available over the counter?',
 '13': 'Can alternative energy effectively replace fossil fuels?',
 '14': 'Is sexual orientation determined at birth?',
 '15': 'Should animals be used for scientific or commercial testing?',
 '16': 'Should prescription drugs be advertised directly to consumers?',
 '17': 'Should recreational marijuana be legal?',
 '18': 'Should churches remain tax-exempt?',
 '19': 'Shoul

## Define the search for the refined prototype with OCR

In [4]:
def search_refined_OCR_prototype(query, num_results):
    body_positive = {
        "from":0,
        "size":num_results,
        "query": {
            "bool":{
                "should":[
                    {"match": { "document_text":query}},
                    {"match": {"ocr_text":{"query":query, "boost":5}}}  # ocr_text sollte einen natürlichen boost besitzen, da die texte viel kürzer sind. Vielleicht muss dieser auch abgeschwächt werden?
                
                ],
                "filter":[
                    {"range": {"sentiment": {"gt": 0}}}
                ]
            }   
        }
    }
    res_positive = es.search(index="boromir_index_dif_clust_meth", body=body_positive)

    body_negative = {
        "from":0,
        "size":num_results,
        "query": {
            "bool":{
                "should":[
                    {"match": { "document_text":query}},
                    {"match": {"ocr_text":{"query":query, "boost":5}}}  # ocr_text sollte einen natürlichen boost besitzen, da die texte viel kürzer sind. Vielleicht muss dieser auch abgeschwächt werden?
                

                ],
                "filter":[
                    {"range": {"sentiment": {"lt": 0}}}
                ]            
            }   
        }
    }
    res_negative = es.search(index="boromir_index_dif_clust_meth", body=body_negative)

    # get ID's from retrieved documents
    result_positive_ids = []
    results_positive = res_positive.get('hits').get('hits')

    for doc in results_positive:
        id = doc.get('_id')
        result_positive_ids.append(id)

    result_negative_ids = []
    results_negative = res_negative.get('hits').get('hits')

    for doc in results_negative:
        id = doc.get('_id')
        result_negative_ids.append(id)

    return result_positive_ids, result_negative_ids

## Make evaluation

In [5]:
all_ranks_pro = []
all_ranks_con = []
numbers = []
true_pos_pro = []
true_pos_con = []
avg_rank_pro = []
avg_rank_con = []

num_results = 10 # number of result id's we want to get

for topic in topicsDic:
    query = topicsDic.get(topic)
    result_ids_pro, result_ids_con = search_refined_OCR_prototype(query, num_results)

    print(query, "\n======================================")

    TP_pro = 0
    TP_con = 0
    total_rank_pro = 0 # the smaller the better
    total_rank_con = 0 # the smaller the better

    # check each result id if it fits the topic (pro)
    for id in result_ids_pro:

        path = define.imagePath() + "/images/"  + id[0:3] + "/" + id + "/pages/"
        file = listdir(path)
        path = path +  file[0] + "/rankings.jsonl"

        lines = []
        for line in open(path, 'r'):
            lines.append(json.loads(line))

        # print(lines[0]['query'])

        if lines[0]['topic'] == topic:
            TP_pro += 1
            rank = lines[0]['rank']
            total_rank_pro += rank

    # check each result id if it fits the topic (con)
    for id in result_ids_con:

        path = define.imagePath() + "/images/"  + id[0:3] + "/" + id + "/pages/"
        file = listdir(path)
        path = path +  file[0] + "/rankings.jsonl"

        lines = []
        for line in open(path, 'r'):
            lines.append(json.loads(line))

        # print(lines[0]['query'])

        if lines[0]['topic'] == topic:
            TP_con += 1
            rank = lines[0]['rank']
            total_rank_con += rank
    
    # print search performance for this topic
        
    print("Number of pictures that fit to the query (Pro): ", TP_pro, "/", num_results)
    print("Precision (Pro): ", TP_pro/num_results)
    print("Total Rank for Query (Pro): ", total_rank_pro)
    print("....................................")
    print("Number of pictures that fit to the query (Con): ", TP_con, "/", num_results)
    print("Precision (Con): ", TP_con/num_results)
    print("Total Rank for Query (Con): ", total_rank_con, "\n")

    all_ranks_pro.append(total_rank_pro)
    all_ranks_con.append(total_rank_con)
    numbers.append(topic)
    true_pos_pro.append(TP_pro)
    true_pos_con.append(TP_con)
    avg_rank_pro.append(round(total_rank_pro/TP_pro, 2))
    avg_rank_con.append(round(total_rank_con/TP_con, 2))

# make final dataframe
df = pd.DataFrame({'Topic': numbers, 'Rank Pro' : all_ranks_pro,  'TP Pro' : true_pos_pro, 'AVG_Rank Pro' : avg_rank_pro, 'Rank Con' : all_ranks_con, 'TP Con' : true_pos_con, 'AVG_Rank Con' : avg_rank_con})
df.to_csv('../'+notebook_name+'.tsv', index = False, sep='\t', line_terminator = '\r\n')

<ipython-input-4-8f73aec82c75>:18: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res_positive = es.search(index="boromir_index_dif_clust_meth", body=body_positive)
C:\Users\miria\anaconda3\lib\site-packages\elasticsearch\connection\base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
<ipython-input-4-8f73aec82c75>:37: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/169

Should teachers get tenure? 
Number of pictures that fit to the query (Pro):  10 / 10
Precision (Pro):  1.0
Total Rank for Query (Pro):  436
....................................
Number of pictures that fit to the query (Con):  9 / 10
Precision (Con):  0.9
Total Rank for Query (Con):  384 

Is vaping with e-cigarettes safe? 
Number of pictures that fit to the query (Pro):  10 / 10
Precision (Pro):  1.0
Total Rank for Query (Pro):  475
....................................
Number of pictures that fit to the query (Con):  10 / 10
Precision (Con):  1.0
Total Rank for Query (Con):  407 

Should insider trading be allowed? 
Number of pictures that fit to the query (Pro):  10 / 10
Precision (Pro):  1.0
Total Rank for Query (Pro):  427
....................................
Number of pictures that fit to the query (Con):  10 / 10
Precision (Con):  1.0
Total Rank for Query (Con):  528 

Should corporal punishment be used in schools? 
Number of pictures that fit to the query (Pro):  10 / 10
Precisi

### create HTML

In [6]:
import define

path_to_images = define.imagePath()

In [7]:
tsv_name = notebook_name+"_"
resultHTML = notebook_name+'.html'

htmlEval = '<!DOCTYPE html> <html>     <head>         <meta charset="UTF-8" name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1">         <title>Evaluation</title> <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">   <link rel="stylesheet" href="myCss.css">      <script src="https://code.jquery.com/jquery-3.2.1.slim.min.js" integrity="sha384-KJ3o2DKtIkvYIK3UENzmM7KCkRr/rE9/Qpg6aAZGJwFDMVNA/GpGFF93hXpG5KkN" crossorigin="anonymous"></script>   <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.12.9/umd/popper.min.js" integrity="sha384-ApNbgh9B+Y1QKtv3Rn7W3mgPxhU9K/ScQsAP7hUibX39j7fakFPskvXusvfa0b4Q" crossorigin="anonymous"></script>    <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/js/bootstrap.min.js" integrity="sha384-JZR6Spejh4U02d8jOt6vLEHfe/JQGiRRSQQxSfFWpi1MquVdAyjUar5+76PVCmYl" crossorigin="anonymous"></script> '
htmlEval = htmlEval + '<script language="Javascript" > \n const dataList = [] \n function addDataToList(query, argpro, argcon, pos, neg) {  var { value } = document.getElementById("user_name").elements["user"]   \n  if(value == ""){ alert ("Please fill in User!")} \n  else{ \n dataList.push(query)    \n var { value } = argpro     \n dataList.push(value)    \n var { value } = argcon     \n dataList.push(value)     \n var { value } = pos   \n dataList.push(value)     \n var { value } = neg  \n dataList.push(value)    \n  var { value } = document.getElementById("user_name").elements["user"]   \n  dataList.push(value)       \n dataList.push(" \\n") \n alert("Inhalt von dataList: "+dataList)  } }  \n function downloadFile() { \n const textFile = btoa("query \\t Argumentative images pro \\t Argumentative images con \\t correct positives \\t correct negatives \\t user_name \\n \\t" + dataList.join("\\t")) \n const saveElement = document.createElement("a") \n saveElement.href = `data:text/plain;base64,${textFile}` \n var { value } = document.getElementById("user_name").elements["user"]   \n  saveElement.download = "' + tsv_name + '"+value+".tsv" \n document.body.appendChild(saveElement) \n saveElement.click() \n document.body.removeChild(saveElement)  } \n </script>'
htmlEval = htmlEval + '</head> <body>   \n  <div class="form-group mb-5">   \n  <input id="download_btn" type="submit" class="btn btn-primary" onClick="downloadFile(); " value="Exportieren" />    \n  </div>  \n  <div class="container-out"> \n  <form name="username" action="" method="post" id="user_name">   \n  <div class="form-group row">    \n  <label for="username" class="col-sm-2 col-form-label">User: </label>    \n  <div class="col-sm-10"> \n  <select multiple class="form-control" name="user">  \n  <option>Gregor</option> \n  <option>Miriam</option> \n  <option>Shirin</option> \n  <option>Thilo</option>  \n  </select></div> \n  </div>  \n  </form>'
num_results = 10


queryListdf = pd.DataFrame(columns = ['topic', 'stance', 'id'])
for tid in topicsDic:
    query = topicsDic.get(tid)
    result_ids_pro, result_ids_con =  search_refined_OCR_prototype(query, num_results)
    src_pro = []
    src_con = []

    for ids in result_ids_pro:
        src_pro.append("images/"  + ids[0:3] + "/" + ids + "/image.png")
    for ids in result_ids_con:
        src_con.append("images/"  + ids[0:3] + "/" + ids + "/image.png")

    link_pro = []
    link_con = []
    #open only directory to get link
    for ids in result_ids_pro:
        with open(path_to_images + "/images/" + ids[0:3] + "/" + ids + "/pages/" + listdir(path_to_images + "/images/" + ids[0:3] + "/" + ids + "/pages")[0] + "/page-url.txt") as f:
            this_link = f.read()
            link_pro.append(this_link)

    for ids in result_ids_con:
        with open(path_to_images + "/images/" + ids[0:3] + "/" + ids + "/pages/" + listdir(path_to_images + "/images/" + ids[0:3] + "/" + ids + "/pages")[0] + "/page-url.txt") as f:
            this_link = f.read()
            link_con.append(this_link)

    #data for HTML processing
    # queryList.append(query)
    evaluation = open("evaluation+link.html").read().format(query=query, 
                                    firstpro = src_pro[0],
                                    secondpro = src_pro[1],
                                    thirdpro = src_pro[2],
                                    fourthpro = src_pro[3],
                                    fifthpro = src_pro[4],
                                    sixthpro = src_pro[5], 
                                    seventhpro = src_pro[6], 
                                    eighthpro = src_pro[7], 
                                    ninthpro = src_pro[8], 
                                    tenthpro = src_pro[9], 
                                    firstcon = src_con[0], 
                                    secondcon = src_con[1], 
                                    thirdcon = src_con[2], 
                                    fourthcon = src_con[3], 
                                    fifthcon = src_con[4], 
                                    sixthcon = src_con[5],
                                    seventhcon = src_con[6],
                                    eighthcon = src_con[7], 
                                    ninthcon = src_con[8], 
                                    tenthcon = src_con[9], 
                                    onepro = link_pro[0], 
                                    twopro = link_pro[1], 
                                    threepro = link_pro[2], 
                                    fourpro = link_pro[3], 
                                    fivepro = link_pro[4], 
                                    sixpro = link_pro[5], 
                                    sevenpro = link_pro[6], 
                                    eightpro = link_pro[7], 
                                    ninepro = link_pro[8], 
                                    tenpro = link_pro[9],
                                    onecon = link_con[0],
                                    twocon = link_con[1], 
                                    threecon = link_con[2], 
                                    fourcon = link_con[3], 
                                    fivecon = link_con[4], 
                                    sixcon = link_con[5], 
                                    sevencon = link_con[6], 
                                    eightcon = link_con[7], 
                                    ninecon = link_con[8], 
                                    tencon = link_con[9])
    #print(topic)
    # for id in result_ids_pro:
    #     # print(id)
    #     queryListdf = queryListdf.append({"topic": query, "stance": "pro", "id": ("/images/"  + id[0:3] + "/" + id + "/pages/")}, ignore_index=True)
    # for id in result_ids_con:
    #     queryListdf = queryListdf.append({"topic": query, "stance": "con", "id": ("/images/"  + id[0:3] + "/" + id + "/pages/")}, ignore_index=True)

    htmlEval = htmlEval + evaluation

    
htmlEval = htmlEval + '</body> </html>'
file = open(resultHTML, 'w')
file.write(htmlEval)
file.close()
    
##debugging/check
#print(queryListdf)
#queryListdf.to_csv("evaluations/"+notebook_name+".tsv",index = False,sep='\t',line_terminator='\r\n')



C:\Users\thilo\AppData\Local\Temp/ipykernel_20180/567046146.py:18: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res_positive = es.search(index="boromir_index_dif_clust_meth", body=body_positive)
C:\Users\thilo\AppData\Local\Temp/ipykernel_20180/567046146.py:37: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res_negative = es.search(index="boromir_index_dif_clust_meth", body=body_negative)
